In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import seaborn as sns

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [2]:
X = pd.read_csv('./dataset-0510/train.csv', index_col='building_id')
X_test = pd.read_csv('./dataset-0510/test.csv', index_col='building_id')

y = X.total_price
X.drop(columns=['total_price'], inplace=True)

## Pipeline RF

In [47]:
# step1. Imputation transformer for completing missing values.
step1 = ('Imputer', Imputer())
# step2. MinMaxScaler
step2 = ('MinMaxScaler', MinMaxScaler())
# step3. feature selection
step3 = ('FeatureSelection', SelectFromModel(RandomForestRegressor()))

finally_step = ('model', RandomForestRegressor(n_estimators=200))

pipeline = Pipeline(steps=[step1, step2, step3, finally_step])

grid = GridSearchCV(pipeline,  
                    param_grid={'model__n_estimators': [200, 300, 400, 500],
                               },
                    cv = 3,
                    scoring = 'neg_mean_absolute_error')
pipeline.fit(X, y)

/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Pipeline(memory=None,
     steps=[('Imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('MinMaxScaler', Normalizer(copy=True, norm='l2')), ('FeatureSelection', SelectFromModel(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', ...tors=200, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [38]:
#print(f"Best model parameters: {grid.best_params_}")
print(f"Best score: {-1 * pipeline.score}")

Best model parameters: {'model__n_estimators': 200}
Best score: 3164773.148295995


In [ ]:
# save test predictions to file
predictions = pipeline.predict(X_test)
output = pd.DataFrame({'building_id': X_test.index, 'total_price': predictions})
output.to_csv('submission/submission.csv', index=False)